**Installation of Dependencies**

In [ ]:
!pip install "arize-phoenix[evals,llama-index]" llama-index-llms-openai openai llama-index-callbacks-arize-phoenix

**Importing Dependencies**

In [1]:
import os
import openai
import dotenv
import pandas as pd
import phoenix as px
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    set_global_handler
)
from llama_index.llms.openai import OpenAI
from tqdm import tqdm

**Running Phoenix in Background to Collect trace data generated by our LlamaIndex app**

In [2]:
session = px.launch_app()

INFO:phoenix.config:📋 Ensuring phoenix working directory: /Users/sachintripathi/.phoenix


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


**Passing OPEN AI API Key**

In [3]:
dotenv.load_dotenv()

True

**Setting 'arize_phoenix' as the global handler. This will mount Phoenix's OpenInferenceTraceCallback as the global handler so as to enable our LLM app tracing**


In [4]:
set_global_handler("arize_phoenix")

**Reading our data directory, building a searchable vector store and initializing query engine for LlamaIndex**

In [5]:
# Loading the data from the directory using SimpleDirectoryReader and building a VectorStoreIndex
# directory name is data and it comprises a text document on the New York City Wikipedia information
documents = SimpleDirectoryReader("./data/").load_data()
vector_index = VectorStoreIndex.from_documents(documents)

# Initialising a query engine
query_engine = vector_index.as_query_engine()

In [6]:
# Passing our queries (prompts) based on the data we have loaded
queries = ["What is Game of Thrones?", "Who is the main character?", "What is the significance of Game of Thrones?", "How did Game of Thrones get its name?"]
queries

['What is Game of Thrones?',
 'Who is the main character?',
 'What is the significance of Game of Thrones?',
 'How did Game of Thrones get its name?']

**Generating responses for our query list using a loop**



In [7]:
# Calling a for-loop to pass our query list's item in the query engine
for query in tqdm(queries):
    query_engine.query(query)

100%|██████████| 4/4 [00:07<00:00,  1.86s/it]


**Importing Phoenix's Evaluation components**

In [8]:
from phoenix.session.evaluation import get_qa_with_reference, get_retrieved_documents
from phoenix.trace import DocumentEvaluations, SpanEvaluations
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    RelevanceEvaluator,
    run_evals,
)

In [9]:
queries_df = get_qa_with_reference(session)
queries_df

,input,output,reference
context.span_id,,,
3d1d46dc047061af,What is Game of Thrones?,A Game of Thrones is a story that takes place ...,{\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...
2ef9d5226c12e4af,Who is the main character?,Ned Stark,{\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...
d78ba54acdaa03e8,What is the significance of Game of Thrones?,"The significance of ""Game of Thrones"" lies in ...",{\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...


In [10]:
retrieved_documents_df = get_retrieved_documents(session)
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
09be194962b316e0 0                  29126f90d06cf4754e6ee4a4d299b448   
                 1                  29126f90d06cf4754e6ee4a4d299b448   
95e7c552f8b4208f 0                  9d3afbb57c508b82eb6cc5b33a023848   
                 1                  9d3afbb57c508b82eb6cc5b33a023848   
9b58739fb4ad488b 0                  5d6bb336dd1781aa9479c9d23669d1d6   
                 1                  5d6bb336dd1781aa9479c9d23669d1d6   
695d400ce534bbdd 0                  4f974dbf07570a87080fed9fc51b029c   
                 1                  4f974dbf07570a87080fed9fc51b029c   

                                                                           input  \
context.span_id  document_position                                                 
09be194962b316e0 0                                      What is Game of Thrones?   
                 1                                      What is Game of Thrones?   
95e7c552f8b4208f 0                                    Who is the main character?   
                 1                                    Who is the main character?   
9b58739fb4ad488b 0                  What is the significance of Game of Thrones?   
                 1                  What is the significance of Game of Thrones?   
695d400ce534bbdd 0                         How did Game of Thrones get its name?   
                 1                         How did Game of Thrones get its name?   

                                                                            reference  \
context.span_id  document_position                                                      
09be194962b316e0 0                  {\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...   
                 1                  As pets, all of the Stark children have direwo...   
95e7c552f8b4208f 0                  {\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...   
                 1                  As pets, all of the Stark children have direwo...   
9b58739fb4ad488b 0                  {\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...   
                 1                  As pets, all of the Stark children have direwo...   
695d400ce534bbdd 0                  {\rtf1\ansi\ansicpg1252\cocoartf2761\n\cocoate...   
                 1                  Robert is mortally wounded in a hunt, and befo...   

                                    document_score  
context.span_id  document_position                  
09be194962b316e0 0                        0.837340  
                 1                        0.816294  
95e7c552f8b4208f 0                        0.756414  
                 1                        0.743353  
9b58739fb4ad488b 0                        0.828202  
                 1                        0.813196  
695d400ce534bbdd 0                        0.848271  
                 1                        0.841167

**Implementing Hallucination, QA Correctness and Relevance Evaluator**

In [11]:
eval_model = OpenAIModel(model_name = "gpt-4o-mini")
hallucination_evaluator = HallucinationEvaluator(eval_model)
qa_correctness_evaluator = QAEvaluator(eval_model)
relevance_evaluator = RelevanceEvaluator(eval_model)

hallucination_eval_df, qa_correctness_eval_df = run_evals(
    dataframe=queries_df,
    evaluators=[hallucination_evaluator, qa_correctness_evaluator],
    provide_explanation=True,
)
relevance_eval_df = run_evals(
    dataframe=retrieved_documents_df,
    evaluators=[relevance_evaluator],
    provide_explanation=True,
)[0]

px.log_evaluations(
    SpanEvaluations(eval_name="Hallucination", dataframe=hallucination_eval_df),
    SpanEvaluations(eval_name="QA Correctness", dataframe=qa_correctness_eval_df),
)
px.log_evaluations(DocumentEvaluations(eval_name="Relevance", dataframe=relevance_eval_df))

WARNI [phoenix.evals.executors] 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


The `model_name` field is deprecated. Use `model` instead.                 This will be removed in a future release.


run_evals |          | 0/6 (0.0%) | ⏳ 00:00<? | ?it/s

WARNI [phoenix.evals.executors] 🐌!! If running inside a notebook, patching the event loop with nest_asyncio will allow asynchronous eval submission, and is significantly faster. To patch the event loop, run `nest_asyncio.apply()`.


run_evals |          | 0/8 (0.0%) | ⏳ 00:00<? | ?it/s

WARNI [phoenix.session.evaluation] This `log_evaluations` function is deprecated and will be removed in a future release. Please use `px.Client().log_evaluations(*evaluations)` instead.
WARNI [phoenix.session.evaluation] This `log_evaluations` function is deprecated and will be removed in a future release. Please use `px.Client().log_evaluations(*evaluations)` instead.
